## Plot PTC postISR image

How to query and plot the postISRCCD images\
Craig Lage - 24Feb23

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.daf.butler import Butler
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm

from lsst.summit.utils import quickSmooth
from lsst.meas.algorithms.installGaussianPsf import InstallGaussianPsfTask
%matplotlib inline

In [ ]:
# The routine below was borrowed from summit_utils, and makes compact plots of an exposure
def plotExp(exp):
    data = quickSmooth(exp.image.array, 1)
    vmin = np.nanpercentile(data, 1)
    vmax = np.nanpercentile(data, 99)
    print(vmin, vmax)
    figure = plt.figure(figsize=(12,12))
    figure.clear()
    ax1 = figure.add_subplot(111)
    im1 = ax1.imshow(data, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
    ax1.tick_params(which="major", direction="in", top=True, right=True, labelsize=8)
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im1, cax=cax)
    plt.tight_layout()
    return figure

In [ ]:
butler = Butler('/repo/main', collections=["LSSTCam/raw/all","LSSTCam/calib","u/cslage/bps_13144S"])
exposure = 3021120300194 # You need some exposure within the PTC run to point the butler at the right run
detector = 55
ptc = butler.get('ptc', detector=detector, exposure=exposure, instrument='LSSTCam')

In [ ]:
# Search for fluxes between 10,000 and 11,000
ampName = 'C00'
fluxMin = 10000.0
fluxMax = 11000.0
for i, flux in enumerate(ptc.rawMeans[ampName]):
    if flux > fluxMin and flux < fluxMax:
        print(flux, ptc.inputExpIdPairs[ampName][i])


In [ ]:
# Now pick one and get the postISRCCD
exposure = 3021120700188
postISRCCD = butler.get('cpPtcProc',  detector=detector, exposure=exposure, instrument='LSSTCam')

In [ ]:
# Now plot it
fig = plotExp(postISRCCD)